## import the libraries 

In [98]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import math
import numpy as np

# Import libraries for running ANOVA and post-hoc test
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats

## Load the data 

In [99]:
df = pd.ExcelFile(r"C:/Users/USER/Downloads/chinese_food.xlsx")

In [100]:
df.sheet_names

['Greenhouse Gas Emission', 'Water Use', 'Land Use']

## Parse the sheets  and create the different dataframes

In [101]:
gas_df = df.parse('Greenhouse Gas Emission')
water_df = df.parse('Water Use')
land_df = df.parse('Land Use')  


In [102]:
gas_df.columns

Index(['Foodcode of Chinese Food Composition Table', 'Food item',
       'Food sub-group', 'Food group',
       'GHGE of 1 kg food as consumed\n(kg CO2-eq/kg)', 'Data source',
       'Language of journal'],
      dtype='object')

## Rename the first and the fifth column

In [103]:
# Rename first column
gas_df = gas_df.rename(columns={gas_df.columns[0]: "Foodcode"})
water_df = water_df.rename(columns={water_df.columns[0]: "Foodcode"})
land_df = land_df.rename(columns={land_df.columns[0]: "Foodcode"})

# Rename 5th column
gas_df = gas_df.rename(columns={gas_df.columns[4]: "co2_released"})
water_df = water_df.rename(columns={water_df.columns[4]: "water_used"})
land_df = land_df.rename(columns={land_df.columns[4]: "land_used"})


## Rename the column “Food group” to “food_group

In [104]:
gas_df = gas_df.rename(columns={gas_df.columns[3]: "food_group"})
water_df = water_df.rename(columns={water_df.columns[3]: "food_group"})
land_df = land_df.rename(columns={land_df.columns[3]: "food_group"})

In [105]:
gas_df

,Foodcode,Food item,Food sub-group,food_group,co2_released,Data source,Language of journal
0,11101,Wheat,wheat,Cereals,0.296835,"Zhang D, Shen J, Zhang F, et al. Carbon footpr...",English
1,11101,Wheat,wheat,Cereals,0.301455,"Zhang G, Wang X, Zhang L, et al. Carbon and wa...",English
2,11101,Wheat,wheat,Cereals,0.331485,"卫凯平,武慧君,黄莉,王晓旭,陈晓芳.农业生产系统氮磷环境影响分析以安徽省为例[J].农业环...",Chinese
3,11101,Wheat,wheat,Cereals,0.340473,宁夏回族自治区平罗县主要农作物碳足迹研究,Chinese
4,11101,Wheat,wheat,Cereals,0.440055,"Wang W, Guo L, Li Y, et al. Greenhouse gas int...",English
...,...,...,...,...,...,...,...
2056,91306,Crock Pot Chicken Soup (Soup),chicken,Poultry,6.840000,refer to the average of food sub-group: chicke...,NaN
2057,91307,Chicken floss,chicken,Poultry,6.840000,refer to the average of food sub-group: chicke...,NaN
2058,91308,Braised chicken (spicy boneless),chicken,Poultry,6.840000,refer to the average of food sub-group: chicke...,NaN
2059,91309,roast chicken,chicken,Poultry,6.840000,refer to the average of food sub-group: chicke...,NaN


## checking missing values

In [106]:
gas_df.isnull().sum(), water_df.isnull().sum(), land_df.isnull().sum()

(Foodcode                  0
 Food item                 0
 Food sub-group            0
 food_group                0
 co2_released              0
 Data source               0
 Language of journal    1021
 dtype: int64,
 Foodcode                  0
 Food item                 1
 Food sub-group            0
 food_group                0
 water_used                0
 Data source               0
 Language of journal    1085
 dtype: int64,
 Foodcode                 0
 Food item                0
 Food sub-group           0
 food_group               0
 land_used                0
 Data source              0
 Language of journal    786
 dtype: int64)

## Droping missing values 

In [107]:
gas_df = gas_df.dropna()
water_df = water_df.dropna()
land_df = land_df.dropna()

## checking dublicates 

In [108]:
gas_df.duplicated().sum(), water_df.duplicated().sum(), land_df.duplicated().sum()

(np.int64(30), np.int64(48), np.int64(1))

## droping missing values 

In [109]:
gas_df = gas_df.drop_duplicates()
water_df = water_df.drop_duplicates()
land_df = land_df.drop_duplicates()


In [113]:
gas_counts = gas_df['food_group'].value_counts()
water_counts = water_df['food_group'].value_counts()
land_counts = land_df['food_group'].value_counts()

side_by_side = pd.concat(
    [gas_counts, water_counts, land_counts], 
    axis=1,
    keys=['gas', 'water', 'land_df']
)
side_by_side


,gas,water,land_df
food_group,,,
Cereals,489,444,32.0
Meat,140,61,11.0
Vegetables,133,101,8.0
Dairy,65,14,12.0
Fruit,55,47,1.0
Poultry,45,19,4.0
Eggs,23,16,2.0
Legumes,15,49,1.0
Aquatic products,13,41,10.0
